In [1]:
import streamlit as st
import pandas as pd
from typing import Dict

from utils import *
from skill_recommendation import get_model, RNNModel
from constants import passive_skill, all_skill, evolve_requirement
import tensorflow as tf

import numpy as np
import os
import time
import pandas as pd

active_skill = [s for s in all_skill if s not in passive_skill and s != 'end']

init_params


In [2]:
import time 

one_step_model = get_model()

In [3]:
def sort_dict_by_value(my_dict):
    sorted_tuples = sorted(my_dict.items(), key=lambda item: item[1])
    sorted_dict = dict(sorted_tuples)
    return sorted_dict

In [5]:
states = None
next_char = ['Revolver']
result = next_char
start = time.time()
skills_to_skip = ['end']

for n in range(10):
    next_char, states, predicted_logits = one_step_model.generate_one_step(next_char, states=states, temperature=0.1)
    result+=(next_char)
    skills_to_skip = get_skills_to_skip(result)
    one_step_model.set_chars_to_skip(skills_to_skip)
    
    print(sort_dict_by_value(transform_skill_list(result)))
    print(skills_to_skip)
    print("-"*15)
            
    
end = time.time()
print(result)
print('\nRun time:', end - start)

{'Revolver': 2}
['end']
---------------
{'Revolver': 3}
['end']
---------------
{'Revolver': 4}
['end']
---------------
{'Revolver': 5}
['end', 'Revolver']
---------------
{'Guard Badge': 1, 'Revolver': 5}
['end', 'Revolver']
---------------
{'Guard Badge': 2, 'Revolver': 5}
['end', 'Revolver']
---------------
{'Guard Badge': 3, 'Revolver': 5}
['end', 'Revolver']
---------------
{'Guard Badge': 4, 'Revolver': 5}
['end', 'Revolver']
---------------
{'Revolver': 5, 'Guard Badge': 5}
['Guard Badge', 'end', 'Revolver']
---------------
{'Drill': 1, 'Revolver': 5, 'Guard Badge': 5}
['Guard Badge', 'end', 'Revolver']
---------------
['Revolver', 'Revolver', 'Revolver', 'Revolver', 'Revolver', 'Guard Badge', 'Guard Badge', 'Guard Badge', 'Guard Badge', 'Guard Badge', 'Drill']

Run time: 0.15007948875427246


In [16]:
import pickle

# with open("./skills", "rb") as fp:
#     all_skill = pickle.load(fp)
#     print(all_skill)
    
ids_from_chars = lambda skills: [all_skill.index(s) for s in skills]
chars_from_ids = lambda ids: [all_skill[i] for i in ids]

In [8]:
df = pd.read_csv('sqllab_untitled_query_7_20230719T124351.csv')
df_filtered = df[(df['result']=='win')]
skills = list(df_filtered['skills'])
skills_dataset = []

for s in skills:
  skills_ind = s.split(',')
  skills_ind = [sk.split('_')[-1] for sk in skills_ind if sk.split('_')[-1] != ""]

  if len(skills_ind) >= 30 and len(skills_ind) <= 65:
    skills_dataset.append(skills_ind)
#     all_skill += skills_ind
# all_skill = list(set(all_skill)) + ["end"]

ids_from_chars = lambda skills: [all_skill.index(s) for s in skills]
chars_from_ids = lambda ids: [all_skill[i] for i in ids]
lengths = [len(s) for s in skills_dataset]
max_length = max(lengths)
dataset_combined = []
for skills in skills_dataset:
  if len(skills) <= max_length:
    to_add = ids_from_chars(skills) + [27] * (max_length - len(skills))
    dataset_combined += to_add
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
ids_dataset = tf.data.Dataset.from_tensor_slices(dataset_combined)
sequences = ids_dataset.batch(max_length)
for seq in sequences.take(1):
  print(chars_from_ids(seq))
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
  print(input_example)
  print(target_example)
  
BATCH_SIZE = 128
BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)


['Soundwave Field', 'Guard Badge', 'Revolver', 'Laser Beam', 'Soundwave Field', 'Drill', 'Wanted Poster', 'Drill', 'Drill', 'Revolver', 'Drill', 'Energy Drink', 'Guard Badge', 'Soundwave Field', 'Revolver', 'Revolver', 'Drill', 'Energy Core', 'Wanted Poster', 'Guard Badge', 'Pocket Watch', 'Laser Beam', 'Captain Shield', 'Laser Beam', 'Guard Badge', 'Soundwave Field', 'Soundwave Field', 'Guard Badge', 'Laser Beam', 'Captain Boot', 'Laser Beam', 'Ammo Thruster', 'Ammo Thruster', 'Captain Shield', 'Captain Boot', 'Captain Shield', 'Captain Boot', 'Captain Shield', 'Wanted Poster', 'Captain Boot', 'Energy Core', 'Captain Shield', 'Wanted Poster', 'Wanted Poster', 'Energy Core', 'Energy Core', 'Ammo Thruster', 'Ammo Thruster', 'Ammo Thruster', 'Pocket Watch', 'Captain Boot', 'Energy Core', 'Pocket Watch', 'Pocket Watch', 'Pocket Watch', 'Energy Drink', 'Energy Drink', 'Energy Drink', 'Energy Drink', 'end', 'end', 'end', 'end', 'end', 'end']
tf.Tensor(
[ 7  9 16 26  7 13 19 13 13 16 13 14  

In [5]:

    
ids_from_chars = lambda skills: [all_skill.index(s) for s in skills]
chars_from_ids = lambda ids: [all_skill[i] for i in ids]

vocab_size = len(all_skill)
embedding_dim = 128
rnn_units = 1024

model = RNNModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [10]:

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()
print(model.summary())

(128, 64, 28) # (batch_size, sequence_length, vocab_size)
Prediction shape:  (128, 64, 28)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.3318782, shape=(), dtype=float32)
Model: "rnn_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  3584      
                                                                 
 gru_1 (GRU)                 multiple                  3545088   
                                                                 
 dense_1 (Dense)             multiple                  28700     
                                                                 
Total params: 3,577,372
Trainable params: 3,577,372
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
new_model = RNNModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
new_model.compile(optimizer='adam', loss=loss)


# Directory where the checkpoints will be saved
new_model.built = True
new_model.load_weights("training_checkpoints/ckpt_150")


for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = new_model(input_example_batch)

    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
    print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
    print("Mean loss:        ", example_batch_mean_loss)
    tf.exp(example_batch_mean_loss).numpy()
    print(new_model.summary())

Prediction shape:  (128, 64, 28)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(0.16281609, shape=(), dtype=float32)
Model: "rnn_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  3584      
                                                                 
 gru_2 (GRU)                 multiple                  3545088   
                                                                 
 dense_2 (Dense)             multiple                  28700     
                                                                 
Total params: 3,577,372
Trainable params: 3,577,372
Non-trainable params: 0
_________________________________________________________________
None


In [84]:
model.built = True
model.load_weights("./training_checkpoints/ckpt_1")

In [85]:

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()
print(model.summary())

(128, 64, 28) # (batch_size, sequence_length, vocab_size)
Prediction shape:  (128, 64, 28)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(5.106539, shape=(), dtype=float32)
Model: "rnn_model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    multiple                  3584      
                                                                 
 gru_10 (GRU)                multiple                  3545088   
                                                                 
 dense_10 (Dense)            multiple                  28700     
                                                                 
Total params: 3,577,372
Trainable params: 3,577,372
Non-trainable params: 0
_________________________________________________________________
None


In [75]:
model = RNNModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
    print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
    print("Mean loss:        ", example_batch_mean_loss)
    tf.exp(example_batch_mean_loss).numpy()
    print(model.summary())

(128, 64, 28) # (batch_size, sequence_length, vocab_size)
Prediction shape:  (128, 64, 28)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.3311408, shape=(), dtype=float32)
Model: "rnn_model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     multiple                  3584      
                                                                 
 gru_8 (GRU)                 multiple                  3545088   
                                                                 
 dense_8 (Dense)             multiple                  28700     
                                                                 
Total params: 3,577,372
Trainable params: 3,577,372
Non-trainable params: 0
_________________________________________________________________
None


In [1]:
new_model

NameError: name 'new_model' is not defined